<a href="https://colab.research.google.com/github/osawa-koki/googlecolab-tester/blob/main/simpleai/v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像データを配列に変換

画像データを分析しやすい配列形式に変換して、保存する処理を担当するブロックです。  

## 前提

ここではすでに画像データの取得と画像データの前処理が行われていることを想定します。  

画像データは「100」枚以上が適正で、その中から学習させるに値しないような画像は含まれていないものとします。  
※ この画像データから不要なデータを取り除く作業はマンパワーが必要で、つらいですが精度の高い人工知能の開発には必要不可欠です。  

## 何をしている???

では、本題です。
ここでは、以下の処理を行っています。

1. 前処理が完了している画像データに対して「リサイズ」「RGB変換」を実行。
2. すべての画像データの配列を格納する包括的な配列にプッシュ。
3. scikit-learnで扱いやすい形式に変換
4. データをトレーニング用と評価用に分類
5. numpy配列ファイルとして保存



In [5]:
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection

# 画像の取得・前処理はすでに終わっているという前提。
# 画像データはdatasetディレクトリ内に格納してある。

# 対象のデータを列挙
classes = ["monkey", "boar", "crow"]
num_classes = len(classes)
image_size = 50

# 画像の読み込み
X = []
Y = []
for index, _class in enumerate(classes):
    # 対象のディレクトリ
    photos_dir = "./dataset/{}".format(_class)
    # 対象の画像一覧を取得
    files = glob.glob(photos_dir + "/*.jpg")
    # 画像を一枚ずつ走査
    for i, file in enumerate(files):
        # 画像を開いて
        image = Image.open(file)
        # 画像をRGB形式に変換して
        image = image.convert("RGB")
        # 画像をリサイズ
        image = image.resize((image_size, image_size))
        # 画像を配列形式に変換して
        data = np.asarray(image, dtype = object)
        # 用意してある配列にプッシュ
        X.append(data)
        Y.append(index)

# scikit-learnで扱いやすい形に変換
X = np.array(X)
Y = np.array(Y)

# 機械学習のトレーニング用と評価用に分類
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y)
xy = (x_train, x_test, y_train, y_test)
np.save("./arraydata/animal.npy", xy)


/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# 畳み込みニューラルネットワーク学習を実行

ここでは前回作成した画像の配列データを実際に学習させます。  
このブロックは以下の関数から構成されます。  

- main
- model_train
- model_eval

それぞれについて簡単に説明します。  

## main

処理モロモロを制御するための関数です。  

## model_train

「The 機械学習」です。
といっても、複雑な数学の知識は不要です。  
全部Keras君がやってくれるからです。  
余力が生じたら、Kerasのドキュメントを読んで、内部構造や詳細な使い方について学習しましょう♪  

ここでは、畳み込みニューラルネットワーク学習の「Sequential」モデルを生成しています。  
おそらく一番簡素な仕組みです。  

## model_eval

モデルの評価を行うための関数です。  
最初に生成したデータを「トレーニング用」と「評価用」に分類しましたが、ここでは「評価用」のデータを使用して、トレーニングされたデータの妥当性を検証します。  





In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import numpy as np
import keras
import tensorflow

# 対象のデータを列挙
classes = ["monkey", "boar", "crow"]
num_classes = len(classes)
image_size = 50

# メイン処理
def main():
    x_train, x_test, y_train, y_test = np.load("./arraydata/animal.npy", allow_pickle=True)
    # 「0-256」から「0-1」の範囲へ変換 
    x_train = x_train.astype("float") / 256
    x_test = x_test.astype("float") / 256
    # 「one-hot-vector」に変換(正解値は1で、それ以外は0)
    # 例えば「monkey」「boar」「crow」を対象に、それがmonkeyであれば[1, 0, 0]、それがboarであれば「0, 1, 0」、それがcrowであれば、「0, 0, 1」
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)

    model = model_train(x_train, y_train)
    model_eval(model, x_test, y_test)

def model_train(x, y):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=x.shape[1:]))
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(3))
    model.add(Activation("softmax"))

    opt = tensorflow.keras.optimizers.RMSprop(lr = 0.0001, decay = 1e-6)
    model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])
    model.fit(x, y, batch_size = 32, epochs = 100)
    model.save("./model/animal_cnn.hs")
    return model

def model_eval(model, x, y):
    scores = model.evaluate(x, y, verbose = 1)
    print("Test loss: {}".format(scores[0]))
    print("Test accuracy: {}".format(scores[1]))
    
if __name__ == "__main__":
    main()



/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/100
21/21 [==============================] - 10s 419ms/step - loss: 0.8938 - accuracy: 0.5837
Epoch 2/100
21/21 [==============================] - 9s 416ms/step - loss: 0.8332 - accuracy: 0.6206
Epoch 3/100
21/21 [==============================] - 9s 417ms/step - loss: 0.8015 - accuracy: 0.6575
Epoch 4/100
21/21 [==============================] - 9s 428ms/step - loss: 0.7931 - accuracy: 0.6836
Epoch 5/100
21/21 [==============================] - 9s 431ms/step - loss: 0.7538 - accuracy: 0.6851
Epoch 6/100
21/21 [==============================] - 10s 467ms/step - loss: 0.7125 - accuracy: 0.7281
Epoch 7/100
21/21 [==============================] - 10s 472ms/step - loss: 0.6587 - accuracy: 0.7634
Epoch 8/100
21/21 [==============================] - 9s 421ms/step - loss: 0.6504 - accuracy: 0.7727
Epoch 9/100
21/21 [==============================] - 9s 421ms/step - loss: 0.6074 - accuracy: 0.7803
Epoch 10/100
21/21 [==============================] - 9s 418ms/step - loss: 0.5858 - acc

7/7 [==============================] - 1s 94ms/step - loss: 1.0095 - accuracy: 0.8203
Test loss: 1.0094572305679321
Test accuracy: 0.8202764987945557


# 実際にAIの評価してもらおう♪

では最後に、AIに画像を与えて、作成したモデルから画像の中の生き物を判断してもらいましょう♪  


In [22]:
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import numpy as np
import keras
import tensorflow
from PIL import Image
from sklearn import model_selection
import sys

# 検査対象の画像
target_img = "target/boar_1.webp"


# 対象のデータを列挙
classes = ["monkey", "boar", "crow"]
num_classes = len(classes)
image_size = 50


def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(50, 50, 3)))
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(3))
    model.add(Activation("softmax"))

    opt = tensorflow.keras.optimizers.RMSprop(lr = 0.0001, decay = 1e-6)
    model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])
    model = load_model("./model/animal_cnn.hs")
    return model

def main():
    image = Image.open(target_img)
    image = image.convert("RGB")
    image = image.resize((image_size, image_size))
    data = np.asarray(image)
    x = []
    x.append(data)
    x = np.array(x)
    model = build_model()
    
    result = model.predict([x])[0]
    predicted = result.argmax()
    percentage = int(result[predicted] * 100)
    # print("{} ({})".format(classes[predicted], percentage))
    print("\n")
    print("{}".format(classes[predicted]))
    print("\n")

if __name__ == "__main__":
    main()



1/1 [==============================] - 0s 94ms/step


monkey


